In [24]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

In [25]:
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_PM_ext.py

GurobiError: Objective Q not PSD (diagonal adjustment of 9.0e+15 would be required)

In [1]:
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekend.py

In [ ]:
def Q_adjust(Q):
    # Ensure Q to be symmetric
    Q = (1.0 / 2) * (Q + np.transpose(Q))

    # Ensure Q to be positive semi-definite
    D, V = LA.eig(Q)
    D = np.diag(D)
    Q_, R_ = LA.qr(V)
    for i in range(0, np.size(Q, 0)):
        if D[i, i] < 0:
            D[i, i] = 1e-10
    Q = np.dot(np.dot(Q_, D), LA.inv(Q_))
    return Q

In [ ]:
def flow_conservation_adjustment(y_0):
    L = len(y_0)  # dimension of flow vector x
    assert(L == 64)

    # y_0 = x[:,1]  # initial flow vector

    model = Model("Flow_conservation_adjustment")

    y = []
    for l in range(L):
        y.append(model.addVar(name='y_' + str(l)))

    model.update() 

    # Set objective: ||y-y_0||^2
    obj = 0
    for l in range(L):
        obj += (y[l] - y_0[l]) * (y[l] - y_0[l])
    model.setObjective(obj)

    # Add nonnegative constraint: y >= 0
    for l in range(L):
        model.addConstr(y[l] >= 0)
    # Add flow conservation constraints
    model.addConstr(y[1] + y[3] == y[0] + y[2])
    model.addConstr(y[0] + y[5] + y[7] == y[1] + y[4] + y[6])
    model.addConstr(y[2] + y[4] + y[9] == y[3] + y[5] + y[8])  # node 3
    model.addConstr(y[6] + y[11] + y[13] + y[15] + y[17] == y[7] + y[10] + y[12] + y[14] + y[16])
    model.addConstr(y[16] + y[21] + y[23] == y[17] + y[20] + y[22])
    model.addConstr(y[8] + y[10] + y[19] == y[9] + y[11] + y[18])  # node 6
    model.addConstr(y[12] + y[14] + y[18] + y[25] + y[27] == y[13] + y[15] + y[19] + y[24] + y[26])
    
    model.addConstr(y[20] + y[31] + y[35] == y[17] + y[23])

    model.update() 

    model.setParam('OutputFlag', False)
    model.optimize()

    y = []
    for v in model.getVars():
        # print('%s %g' % (v.varName, v.x))
        y.append(v.x)
    # print('Obj: %g' % obj.getValue())
    return y